## 张量的赋值与索引

In [1]:
import torch

b = torch.ones(3)   # 创建大小为3的一维张量, 用1.0填充
print(b)

print(b[1])
print(float(b[1]))  # 直接索引元素得到的还是张量, 需要转化成浮点型

tensor([1., 1., 1.])
tensor(1.)
1.0


In [14]:
b[2] = 2.0
print(b)
b[2] = torch.tensor(3.)
print(b)
b[2] = torch.tensor([4.])     # 把 python list 转化成张量
# ERROR: b[2] = torch.tensor([4., 5.])
print(b)

# 三种赋值方式等价

tensor([1., 1., 2.])
tensor([1., 1., 3.])
tensor([1., 1., 4.])


## 张量的形状

In [15]:
points = torch.tensor([[4.0, 1.0], [5.0, 2.0], [6.0, 3.0]])
print('Get shape: ', points.shape)   # 3行, 2列

print(points[0,1])   # 双索引, 0号列表里的1号元素, 从外往里数方括号

Get shape:  torch.Size([3, 2])
tensor(1.)


复习python列表索引

In [16]:
some_list = list(range(6))
print(some_list[:])
print(some_list[1:4])
print(some_list[:4])
print(some_list[:-1])
print(some_list[1:5:2])   # 1到5, 左闭右开, 步长为2

[0, 1, 2, 3, 4, 5]
[1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[1, 3]


## 张量维度的命名

In [17]:
img_t = torch.randn(3, 5, 5)        # Shape (channels, rows, columns)
print(img_t.shape, img_t)
batch_t = torch.randn(2, 3, 5, 5)   # Shape (batch, channels, rows, columns <- 这个叫作第四个维度的数量)
# print('batch_t: ',batch_t)

img_gray_naive = img_t.mean(-3)
# 消除倒数第三个维度. 假设这个维度的数量是3, 把这个维度上的每3个值叠起来用一个平均值代替. 
batch_gray_naive = batch_t.mean(-3)
print(img_gray_naive.shape)
# print(batch_gray_naive.shape, batch_gray_naive)

torch.Size([3, 5, 5]) tensor([[[-2.1124, -0.1553,  0.4629, -1.2841, -0.7911],
         [-0.4554, -0.1023,  1.2334,  0.5040,  1.0239],
         [ 1.5715,  0.4889,  0.4412, -1.8039, -0.1684],
         [-0.6147, -0.8977, -1.0812, -1.7793, -2.1803],
         [-0.5545, -0.5752,  0.7825,  1.2249,  0.1197]],

        [[ 0.0218,  0.2895,  0.0707,  0.0917, -0.3342],
         [ 0.4822,  1.1502, -1.5371,  0.7558,  0.2984],
         [ 1.7904, -1.0492, -0.3073,  0.0193, -0.6199],
         [ 0.8969, -1.6550, -0.2114, -2.7102, -0.1010],
         [-0.4362, -1.2606, -0.8926, -0.6409,  0.3925]],

        [[ 0.8049,  0.9331, -0.9726,  0.5734,  0.7047],
         [-0.9811, -0.4129,  1.0603,  0.0539,  0.8855],
         [-0.0255,  0.1353,  0.7175, -1.2731, -1.2798],
         [-1.8609,  0.4330, -2.4583, -0.2989, -0.8667],
         [ 0.7479, -0.6694,  0.1192,  0.6350,  0.3875]]])
torch.Size([5, 5])


```
升维
torch.unsqueeze(
	input, 
	dim, 	     # dim 从0算起, 将要扩增的维度【必填】
	out = None)
如果dim为负, 则将会被转化dim+input.dim()+1, 即在最后增一维

降维
torch.squeeze(
	input, 
	dim, 	     # dim 从0算起, 将要挤压的维度【必填】
	out = None)
如果dim指定的维度的值为1, 则将该维度删除, 若指定的维度值不为1, 则返回原来的tensor

unsqueeze_ 和 unsqueeze 实现一样的功能, 
区别在于 unsqueeze 不会对使用 unsqueeze 的 tensor 进行改变, 想要获取 unsqueeze 后的值必须赋予个新值, 
unsqueeze_ 则会对自己改变. 
```

```
张量点乘, 对于某个维度:
若数量都不是1, 则必须数量相等才能相乘, 计算方法是: i位对i位相乘
若数量有一个是1, 则可以相乘, 计算方法是: 多数量的每一个值分别去乘单数量的值, 类似乘法分配律
```

In [18]:
weights = torch.tensor([0.2126, 0.7152, 0.0722])
# 我们可以对相同形状的张量进行乘法运算, 所以需要对张量进行升维/降维

unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)  # 在最后(即最小维度)升两维
print(unsqueezed_weights.shape, unsqueezed_weights)

img_weights = (img_t * unsqueezed_weights)
print(img_weights.shape)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
# batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

torch.Size([3, 1, 1]) tensor([[[0.2126]],

        [[0.7152]],

        [[0.0722]]])
torch.Size([3, 5, 5])


In [15]:
# tensor multiplication test

def mult(A, B):
    print(A, '<TIMES>', B, ' = ', A * B)

x = torch.tensor([3, 5, 7, 9])
# ERROR: x = torch.tensor([3, 5, 7])
y = torch.tensor([13, 17, 19, 23])
mult(x, y)

x = torch.tensor([3])
y = torch.tensor([13, 17, 19, 23])
mult(x, y)

x = torch.tensor([[3]])
x = torch.tensor([[3], [27]])
# ERROR: x = torch.tensor([[3], [27], [99]])
y = torch.tensor([[13, 17, 19, 23], [1, 5, 7, 11]])
mult(x, y)

x = torch.tensor([[3, 4, 1, 2], [5, 6, 7, 8]])
y = torch.tensor([[10, 11, 12, 13], [14, 15, 16, 17]])
mult(x, y)

tensor([3, 5, 7, 9]) <TIMES> tensor([13, 17, 19, 23])  =  tensor([ 39,  85, 133, 207])
tensor([3]) <TIMES> tensor([13, 17, 19, 23])  =  tensor([39, 51, 57, 69])
tensor([[ 3],
        [27]]) <TIMES> tensor([[13, 17, 19, 23],
        [ 1,  5,  7, 11]])  =  tensor([[ 39,  51,  57,  69],
        [ 27, 135, 189, 297]])
tensor([[3, 4, 1, 2],
        [5, 6, 7, 8]]) <TIMES> tensor([[10, 11, 12, 13],
        [14, 15, 16, 17]])  =  tensor([[ 30,  44,  12,  26],
        [ 70,  90, 112, 136]])
